In [1]:
import pandas as pd
from GPTModel import GPTInference

In [2]:
def read_file(path):
    return pd.read_csv(path)

In [8]:
def get_atomic_facts_gpt(gpt_model_atomic, text, text_type):
    instr = f'Segment the following {text_type} into important facts'
    prompt = f'{instr}\nDialogue: {text}'
    print(prompt)
    gpt_response = gpt_model_atomic.get_chatgpt_response(prompt)
    texts = gpt_response.split('\n')
    texts = [re.sub('[1-9]', '', each) for each in texts]
    texts = [each.strip(string.punctuation).strip() for each in texts]
    return texts

In [9]:
def mask_all_errtypes(text):
    doc = nlp(text)
    origin_tokens = [token for token in doc]
    all_masked_results = []
    mask_idx = []
    mask_tokens = []
    for idx, token in enumerate(doc):
        append = False
        if 'subj' in token.dep_ or 'obj' in token.dep_:
            # print(idx, origin_tokens)
            # masked_text = [each.text for each in origin_tokens[:idx]] + ['<BLANK>'] + [each.text for each in origin_tokens[idx+1:]]
            # masked_text = ' '.join(masked_text)
            # print(masked_text)
            append = True
            token_type = 'subjobj'
            
        elif 'VERB' in token.pos_:
            append = True 
            token_type = 'verb'

        if append:
            mask_idx.append(idx)
            mask_tokens.append((idx, token.text, token_type))
    # print([])
    print(mask_tokens)  

    error_types_indices_map = {}
    for token_idx, token_text, err_type in mask_tokens:
        if err_type not in error_types_indices_map:
            error_types_indices_map[err_type] = []
        error_types_indices_map[err_type] += [token_idx]
        
    for err_type, err_indices in error_types_indices_map.items():
        for idx in err_indices:
            masked_text = [each.text for each in origin_tokens[:idx]] + ['<BLANK>'] + [each.text for each in origin_tokens[idx+1:]]
            masked_text = ' '.join(masked_text)
            all_masked_results.append((masked_text, origin_tokens[idx].text, err_type))
    return all_masked_results

In [10]:
def get_subjobj_answer(gpt_model_corruptor, dlg, text, answer):
    instruction = "List other subject or object entities from the dialogue that can be used to fill in the <BLANK> in the summary"
    # instruction = "List subject or object entities that can be used to fill in the <BLANK>"
    prompt_template = f'Dialogue: {dlg}\nSummary: {text}\n{instruction}\nAnswer: {answer},'
    # print(prompt_template)
    # print('--')
    # prompt_template = f'Summary: {text}\n{instruction}\nAnswer: {answer},'
    answer = gpt_model_corruptor.get_chatgpt_response(prompt_template)
    return answer

In [11]:
df = read_file('/home/sanjana/factual_evaluation_source_based/datasets/sota_annotations/dialogue_aggrefact.csv')

In [12]:
text = df['Summary'].values[10]
dlg = df['Dialogue'].values[10]

In [14]:
import re, string
gpt_model_atomic = GPTInference()
texts = get_atomic_facts_gpt(gpt_model_atomic, dlg, 'dialogue')
texts

Segment the following dialogue into important facts
Dialogue: Greg: Hi, honey. I need to stay after hours :-(
Betsy: Again?
Greg: I'm sorry!
Betsy: What about Johnny?
Greg: Well, could you pick him up? 
Betsy: What if I can't?
Greg: Betsy?
Betsy: What if I can't?
Greg: Can't you, really?
Betsy: I can't. Today I need to work long hours as well. Tuesdays are your days in the kindergarten.
Greg: Talk to you later. I'll see what I can do.
Betsy: You'd better think of something.
Greg: Oh. Just stop it now.


['Greg needs to stay after hours at work',
 'Betsy also needs to work long hours today',
 "Today is Greg's day to pick up Johnny from kindergarten",
 'Betsy cannot pick up Johnny today',
 'Greg needs to find a solution to pick up Johnny']

In [ ]:
instruction = ''